In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#main directory 
rrot_path=/content/drive/My\ Drive/kaggle/cancer/train

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from glob import glob
from random import shuffle
import cv2
import matplotlib.pyplot as plt


In [0]:
#print(os.listdir('/content/drive/My Drive/CIs'))
print(os.listdir('/content/drive/My Drive/kaggle/cancer'))
#print(os.listdir('/content/drive/My Drive/kaggle/cancer'))
!ls /content/train

['train_labels.csv.zip', 'sample_submission.csv.zip', 'train', 'Cancer.ipynb', 'pcamv1']


In [0]:
df_train = pd.read_csv('/content/drive/My Drive/kaggle/cancer/train_labels.csv.zip')
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}
df_train.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [0]:
df_train['label'].value_counts()

0    130908
1     89117
Name: label, dtype: int64

In [0]:
def readImage(path):
    # OpenCV reads the image in bgr format by default
    bgr_img = cv2.imread(path)
    # We flip it to rgb for visualization purposes
    b,g,r = cv2.split(bgr_img)
    rgb_img = cv2.merge([r,g,b])
    return rgb_img
#readImage('409c141ab6ef2dfd08c179e13ab3a65fd6cee983'+'.tif')



In [0]:
## Just displaying the data 
###########################3
##plotting  negative samples

train_imgs = os.listdir("/content/train")
shuffled_data = shuffle(df_train)
fig, ax = plt.subplots(2,5, figsize=(20,8))
fig.suptitle('Histopathologic scans of lymph node sections',fontsize=20)
# Negatives
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 0]['id'][:5]):
    #path = os.path.join(train_imgs[i], idx)
    print(i)
    print(idx)
    ax[0,i].imshow(readImage(idx +'.tif'))
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='b',facecolor='none', linestyle=':', capstyle='round')
    ax[0,i].add_patch(box)
ax[0,0].set_ylabel('Negative samples', size='large')
#####################
## Plotting positive samples
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 1]['id'][:5]):
    ax[1,i].imshow(readImage(idx +'.tif'))
    # Create a Rectangle patch
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='r',facecolor='none', linestyle=':', capstyle='round')
    ax[1,i].add_patch(box)
ax[1,0].set_ylabel('Tumor tissue samples', size='large')

In [0]:
# Bad images in training set i.e either too dark or too white 

def bad_image(shuffleddata):
  
  too_dark_idx = []
  too_bright_idx = []

  x_tot = np.zeros(3)
  x2_tot = np.zeros(3)
  counted_ones = 0
  for i, idx in enumerate(shuffleddata['id']):
      imagearray = readImage(idx + '.tif')
      if(imagearray.max() < 10/255):
          too_dark_idx.append(idx)
          continue # do not include in statistics
      # is this too bright
      if(imagearray.min() > 245/255):
          too_bright_idx.append(idx)
          continue # do not include in statistics

  print('There was {0} extremely dark image'.format(len(too_dark_idx)))
  print('and {0} extremely bright images'.format(len(too_bright_idx)))
  print('Dark one:')
  print(too_dark_idx)
  print('Bright ones:')
  print(too_bright_idx)
  return too_dark_idx,too_bright_idx

dark,bright=bad_image(shuffled_data)

## The too bright images were not completely white . Instead the lymph node stains were very light coloured . So we cannot delete those.
## Plain white image was only 1 which was dd6dfed324f9fcb6f93f46f32fc800f2ec196be2.tif





There was 1 extremely dark image
and 46798 extremely bright images
Dark one:
['9369c7278ec8bcc6c880d99194de09fc2bd4efbe']
Bright ones:
['...']


In [0]:

fig, ax = plt.subplots(2,len(dark), figsize=(25,9))
fig.suptitle('Almost completely black or white images',fontsize=20)

# Too dark
#print(readImage(dark+'tif'))
i = 0
for idy in len(dark):
    print(idy)
    img=dark[idy]
    print(img)
    lbl = shuffled_data[shuffled_data['id'] == dark[idy]['label'].values[0]
    
    #path = os.path.join('/content', img)
    #print(path)
    #idx = ''.join(idx)
    
    ax[0,i].imshow(readImage(idy+'tif'))
    ax[0,i].set_title(img + '\n label=' + str(lbl), fontsize = 8)
    i += 1
ax[0,0].set_ylabel('Extremely dark images', size='large')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import shutil


import os
import pandas as pd
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt
from keras.models import load_model,save_model
from keras.callbacks import ModelCheckpoint
from matplotlib.patches import Rectangle
import os
from scipy.misc import imsave
from tqdm import tqdm

import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, merge, GlobalAveragePooling2D, MaxPool2D, GlobalMaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input, Concatenate
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
import keras.backend as K
from math import sqrt
from keras.callbacks import History 

from keras.applications import ResNet50, VGG19, NASNetMobile,DenseNet169
import gc




In [ ]:
trainPath = '../cancer/train/'
df = pd.read_csv("../cancer/train_labels.csv")

#Exclude Plain white image
df = df[df['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# Exclude the black images 
df = df[df['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']




lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, epsilon=1e-5, patience=3, verbose=1)

modelID=0
checkPoint = ModelCheckpoint(filepath='./model_resnet_{epoch:02d}_{val_loss:.4f}_{val_acc:4f}.hdf5',verbose=1,save_best_only=True,mode=min)
bestCheckPoint = ModelCheckpoint(filepath='./bestModel.hdf5',verbose=0,save_best_only=True,mode=min)

datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.1)
                            
trainGen=datagen.flow_from_dataframe(df, trainPath, x_col='id',y_col='label',has_ext=False,target_size=(96,96),subset='training', class_mode='binary',shuffle=True)
valGen = datagen.flow_from_dataframe(df, trainPath, x_col='id',y_col='label',has_ext=False,target_size=(96,96),subset='validation', class_mode='binary',shuffle=True)

def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act
    
    
model = DenseNet169(include_top = False, input_shape=(96,96,3))

# for layer in model.layers:
#     layer.trainable = False


x = model.output

out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
# out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2])
out = Dropout(0.4)(out)
out = Dense(256, activation="relu")(out)
predictions = Dense(1, activation="sigmoid")(out)




fullModel = Model(inputs=model.input, outputs=predictions)

adam = Adam(lr=0.00001)

fullModel.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])



fullModel.fit_generator(trainGen,epochs=14,steps_per_epoch=2000,validation_data=valGen, validation_steps=100,callbacks=[lr_reduce,checkPoint,bestCheckPoint])

fullModel.load_weights('./bestModel.hdf5')


base_test_dir = '../input/test/'
test_files = glob(os.path.join(base_test_dir,'*.tif'))
submission = pd.DataFrame()
file_batch = 5000
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[3].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = (K_test - K_test.mean()) / K_test.std()
    predictions = fullModel.predict(K_test)
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[["id", "label"]]])
submission.head()

submission.to_csv("submission.csv", index = False, header = True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
#####
#    This function prints and plots the confusion matrix.
#    Normalization can be applied by setting `normalize=True`.
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

    
test_labels = test_gen.classes
confmat = confusion_matrix(test_labels, predictions.argmax(axis=1))
plot_labels = ['no_tumor_tissue', 'has_tumor_tissue']

plot_confusion_matrix(confmat, plot_labels, title='Confusion Matrix')